In [2]:
import cv2

In [ ]:
import os
import cv2

def generate_dataset():
    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    if face_classifier.empty():
        print("Error loading cascade classifier.")
        return

    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            cropped_face = img[y:y+h, x:x+w]
            return cropped_face  # Return the first detected face

    cap = cv2.VideoCapture(0)
    user_id = int(input("Enter user ID: "))
    max_samples = 150
    img_id = 0

    if not os.path.exists("data"):
        os.makedirs("data")

    print("Collecting samples...")

    while img_id < max_samples:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame.")
            break
        
        face = face_cropped(frame)
        if face is not None:
            img_id += 1
            face = cv2.resize(face, (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            file_name_path = f"data/user.{user_id}.{img_id}.jpg"
            cv2.imwrite(file_name_path, face)

            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Cropped face", face)
        else:
            print("No face detected.")

        # Show the frame with potential detections
        cv2.imshow("Frame", frame)

        if cv2.waitKey(1) == 13:  # Enter key
            break

    cap.release()
    cv2.destroyAllWindows()
    
    print("Collecting samples is completed. Total samples collected:", img_id)

generate_dataset()


In [2]:
import os
import numpy as np
import cv2
from PIL import Image

def train_classifier(data_dir, output_file="classifier.xml"):
    # List all image files in the data directory
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    
    faces = []
    ids = []
    
    # Iterate over all files in the directory
    for image_path in path:
        try:
            # Open the image and convert it to grayscale
            img = Image.open(image_path).convert('L')
            image_np = np.array(img, 'uint8')
            
            # Extract the ID from the filename (assumes format "name.id.jpg")
            file_name = os.path.basename(image_path)
            id = int(file_name.split(".")[1])  # Adjust according to your filename structure
            
            # Append the face image and ID
            faces.append(image_np)
            ids.append(id)
        
        except Exception as e:
            print(f"Error processing {image_path}: {e}")
    
    if not faces or not ids:
        print("No valid faces found. Training aborted.")
        return
    
    ids = np.array(ids)
    
    # Initialize and train the LBPH face recognizer
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    
    # Save the trained classifier to a file
    clf.write(output_file)
    print(f"Classifier trained and saved to {output_file}")

# Call the function with your data directory
if __name__ == "__main__":
    train_classifier("data")


Classifier trained and saved to classifier.xml


In [ ]:
import cv2
import numpy as np

def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
     
    for (x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
         
        id, pred = clf.predict(gray_img[y:y + h, x:x + w])
        confidence = int(100 * (1 - pred / 300))
         
        if confidence > 70:
            name = "UNKNOWN"
            if id == 1:
                name = "Rajesh" 
            elif id == 2:
                name = ""
            elif id == 3:
                name = ""
            elif id == 4:
                name = ""
            elif id == 5:
                name = ""
            elif id == 6:
                name = ""
            cv2.putText(img, name, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else:
            cv2.putText(img, "UNKNOWN", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)
     
    return img

# Load classifier
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")
 
# Start video capture
video_capture = cv2.VideoCapture(0)

try:
    while True:
        ret, img = video_capture.read()
        if not ret:
            print("Failed to grab frame")
            break
        img = draw_boundary(img, faceCascade, 1.3, 6, (255, 255, 255), "Face", clf)
        cv2.imshow("Face Detection", img)

        if cv2.waitKey(1) & 0xFF == ord('a'):
            break
finally:
    video_capture.release()
    cv2.destroyAllWindows()
    print("Exiting the program.")

Exiting the program.
